# W02a Online Property Resale Platform - Python and Sqlite3

A company would like to implement a web application to manage the properties in the resale market online.

The following information of each `User` is stored:
- `UserID` – unique string to identify each user. User can be a seller or a buyer.
- `Name` – name of the user
- `Contact` – contact of the user
- `Email` – email of the user

The following information of each `Property` is stored:
- `PropertyID` – unique string to identify the property
- `Address` – address with street names and property name
- `Postal` – postal code
- `TotalArea` – total area in terms of square meter
- `NoOfBedroom` – no. of bedrooms
- `NoOfToilet` – no. of toilets
- `AskingPrice` – asking price
- `Status` – sold status, default as 'False'

The following information of each `Record` is stored:  
Note:`Records` may only have `SellerID`, `PropertyID` and `DateListed` if it is not yet sold
- `RecordID` – unique autoincrement integer to identify a potential resale record. 
- `SellerID` – seller id
- `PropertyID` – property id
- `DateListed` – date which the property is being listed for sale
- `BuyerID` – buyer id
- `SoldPrice` – final transaction price
- `SoldDate` – date which the property is sold


The information is to be stored in the following tables:
- `User`
- `Property`
- `Record`


In [1]:
# Run to link to db and unlock db when needed
import sqlite3
import csv

# !NOTE: if "db.sqlite3" not exist, it will be created
db_file = "property.db"
conn = sqlite3.connect(db_file)
conn.close()

### Q1
Use DB Browser for SQLite to create the database schema for the tables `User`, `Property`, and `Record`.

Save the database as `property.db`.

Copy and save the SQL script used to create the tables in the cell below.


In [3]:
# Q1 Answer:
query1 = """
CREATE TABLE IF NOT EXISTS "User" (
	"UserID"	TEXT NOT NULL,
	"Name"	TEXT NOT NULL,
	"Contact"	INTEGER NOT NULL,
	"Email"	TEXT NOT NULL,
	PRIMARY KEY("UserID")
);
"""

query2 = """
CREATE TABLE IF NOT EXISTS "Property" (
	"PropertyID"	TEXT NOT NULL,
	"Address"	TEXT NOT NULL,
	"Postal"	INTEGER NOT NULL,
	"TotalArea"	INTEGER NOT NULL,
	"NoOfBedroom"	INTEGER NOT NULL,
	"NoOfToilet"	INTEGER NOT NULL,
	"AskingPrice"	REAL NOT NULL,
	"Status"	TEXT NOT NULL DEFAULT 'FALSE' CHECK("Status" IN ('TRUE', 'FALSE')),
	PRIMARY KEY("PropertyID")
);
"""

query3 = """
CREATE TABLE IF NOT EXISTS "Record" (
	"RecordID"	INTEGER NOT NULL,
	"SellerID"	TEXT NOT NULL,
	"PropertyID"	TEXT NOT NULL,
	"DateListed"	TEXT NOT NULL,
	"BuyerID"	TEXT,
	"SoldPrice"	REAL,
	"SoldDate"	TEXT,
	PRIMARY KEY("RecordID" AUTOINCREMENT)
);
"""

queries = [query1, query2, query3]


#### Q2

Write python code to create the database `property.db` and the tables `User`, `Property`, and `Record` in the database.
* Take note the separator is ';' instead of commas.

In [ ]:
# Q2 Answer:
conn = sqlite3.connect(db_file)
for query in queries:
    conn.execute(query)
conn.commit()
conn.close()

#### Q3
Read data from `users.csv`, `properties.csv` and `records.csv` in `data_files` folder, and insert the data into the respective tables.

In [31]:
# Q3 Answer:

def read_csv_to_db(table, file_name):
    with open(f'data_files/{file_name}', "r") as f:
        csv_reader = csv.reader(f, delimiter=';')
        header = next(csv_reader)
        # print(header)
        data = []
        for row in csv_reader:
            data.append(row)
        # print(data)
    
    conn = sqlite3.connect(db_file)
    ph = ",".join(["?" for _ in header])
    query = f"""
    INSERT INTO {table}
    {tuple(header)}
    VALUES
    ({ph})
    """
    # print(query)

    for row in data:
        conn.execute(query, tuple(row))

    conn.commit()
    conn.close()

files = {"Property": "properties.csv",
        "Record": "records.csv",
        "User": "users.csv"}

for table in files:
    file_name = files[table]
    read_csv_to_db(table, file_name)

#### Q4

List out all unsold properties in the system which has 3 bedrooms and 3 toilets, sorted according to price in ascending order, with the following fields selected.

`RecordID, PropertyID, Address, TotalArea, NoOfBedroom, NoOfToilet, AskingPrice`

In [36]:
# Q4 Answer:

conn = sqlite3.connect(db_file)
query = """
SELECT Record.RecordID, Property.PropertyID, Property.Address, Property.TotalArea, 
    Property.NoOfBedroom, Property.NoOfToilet, Property.AskingPrice
FROM Property
INNER JOIN Record ON Property.PropertyID = Record.PropertyID
WHERE Property.Status = 'FALSE' AND Property.NoOfBedroom = 3 AND Property.NoOfToilet = 3
ORDER BY Property.AskingPrice ASC;
"""
cursor = conn.execute(query)
print(cursor.fetchall())
cursor.close()
conn.close()

[(11, 'Property006', '514 chai chee lane', 108, 3, 3, '530,000.00'), (1, 'Property007', '77 Commonwealth Drive', 106, 3, 3, '749,000.00')]


#### Q5

List out all the properties that are sold in the system, with the following fields selected.

`RecordID, PropertyID, Address, TotalArea, NoOfBedroom, NoOfToilet, AskingPrice, SoldPrice, SoldDate`

In [39]:
# Q5

conn = sqlite3.connect(db_file)
query = """
SELECT Record.RecordID, Property.PropertyID, Property.Address, Property.TotalArea, 
    Property.NoOfBedroom, Property.NoOfToilet, Property.AskingPrice, Record.SoldPrice, Record.SoldDate
FROM Property
INNER JOIN Record ON Property.PropertyID = Record.PropertyID
WHERE Property.Status = 'TRUE'
ORDER BY Record.SoldDate ASC;
"""
cursor = conn.execute(query)
print(cursor.fetchall())
cursor.close()
conn.close()

[(10, 'Property002', '141 Market Street', 90, 2, 2, '919,000.00', '910,000.00', '20200510'), (2, 'Property003', '27 Wilby Road', 110, 3, 3, '703,000.00', '692,000.00', '20200605'), (5, 'Property004', '9 penang road', 100, 3, 3, '1,046,000.00', '1,035,000.00', '20200620')]


#### Q6

Define a function `update_property` to update the asking price of a property with a given `property_id`, to a new price, `new_price`.

In [40]:
# Q6 Answer:

def update_property(property_id, new_price):
    conn = sqlite3.connect(db_file)
    query = """
    UPDATE Property
    SET AskingPrice = ?
    WHERE PropertyID = ?;
    """
    conn.execute(query, (new_price, property_id))
    conn.commit()
    conn.close()